In [2]:
from fastai.metrics import accuracy
from fastai.optimizer import OptimWrapper
from fastai.vision.all import *
from PIL import Image

from torch import optim
import torch.nn as nn
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from transformers import AdamW
import cv2

import gc
import glob
import inspect
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
from functools import partial

from skimage.metrics import structural_similarity as ssim
from skimage import color, transform
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings('ignore')
set_seed(3, reproducible=True)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
path = '/kaggle/input/zindidata/'

In [4]:
trn_path = f'{path}images/images/'

In [5]:
train            = pd.read_csv(f'{path}Train.csv')
test             = pd.read_csv(f'{path}Test.csv')
SampleSubmission = pd.read_csv(f'{path}SampleSubmission.csv')

In [6]:
Train = train.copy()
Test = test.copy()

skf = StratifiedKFold(5, shuffle=True, random_state=3)
X = Train.drop(columns='damage')
y = Train.damage

for fold, (_, valid_index) in enumerate(skf.split(X, y)):
  Train.loc[valid_index, "fold"] = fold

Train.fold = Train.fold.astype(int)

In [7]:
damage = Train.damage.unique()

In [8]:
class_weights = torch.FloatTensor([1.3, 1.2, 1.4, 1.0, 2.8]).cuda()

In [9]:
IMG_HEIGHT = IMG_WIDTH = 224
IMAGENET_MEAN = (0.45, 0.46, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)
INCEPTION_MEAN = INCEPTION_STD = (0.5, 0.5, 0.5)


class AlbumentationsTransform(RandTransform):
    "A transform handler for multiple `Albumentation` transforms"
    split_idx, order = None, 2
    def __init__(self, train_aug, valid_aug): store_attr()

    def before_call(self, b, split_idx):
        self.idx = split_idx

    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']

        return aug_img


def get_train_transforms(mean_std):
    augmentations = [
        A.VerticalFlip(p=.4),
        A.HorizontalFlip(p=.5),
        A.RandomRotate90(p=.2),
        A.ShiftScaleRotate(
            shift_limit=0.2, scale_limit=0.3,
            rotate_limit=45, border_mode=0, p=.6
        ),
        A.ImageCompression(quality_lower=99, quality_upper=100),
        A.RandomBrightnessContrast(brightness_limit=(-0.15,0.2), contrast_limit=(-0.1, 0.1), p=0.5),
        A.Cutout(
            max_h_size=int(IMG_HEIGHT*0.3),
            max_w_size=int(IMG_WIDTH*0.3),
            num_holes=4,
            p=.7,
        ),
    ]
    if mean_std=='imagenet':
        augmentations.append(A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD))
    elif mean_std=='inception':
        augmentations.append(A.Normalize(mean=INCEPTION_MEAN, std=INCEPTION_STD))
    else:
        augmentations.append(A.Normalize(mean=0, std=1))

    augmentations.append(ToTensorV2())
    return A.Compose(augmentations)


def get_valid_transforms(mean_std):
    augmentations = []
    if mean_std=='imagenet':
        augmentations.append(A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD))
    elif mean_std=='inception':
        augmentations.append(A.Normalize(mean=INCEPTION_MEAN, std=INCEPTION_STD))
    else:
        augmentations.append(A.Normalize(mean=0, std=1))

    augmentations.append(ToTensorV2())
    return A.Compose(augmentations)

def get_item_tfms(mean_std='imagenet'):
    return [Resize(224, method='squish'), AlbumentationsTransform(get_train_transforms(mean_std), get_valid_transforms(mean_std))]

In [10]:
def get_dls(fold=0, mean_std='imagenet'):
    clas_block = DataBlock(blocks = (ImageBlock, CategoryBlock(vocab=damage)),
                       get_x = ColReader('filename', pref = trn_path),
                       get_y = ColReader('damage'),
                       splitter = MaskSplitter(Train.fold == fold),
                       item_tfms = get_item_tfms(mean_std), #Resize(320, method = 'squish'),
                       batch_tfms = aug_transforms(min_scale= 0.75), #aug_transforms(size = 128, min_scale = 0.75)
                      )
    dls = clas_block.dataloaders(Train, bs=64)
    dls.rng.seed(3)

    return dls

In [11]:
def free_memory(to_delete: list):
    calling_namespace = inspect.currentframe().f_back

    for _var in to_delete:
        calling_namespace.f_locals.pop(_var, None)
        gc.collect()
        torch.cuda.empty_cache()

In [12]:
timm.list_models(pretrained=True)

['bat_resnext26ts.ch_in1k',
 'beit_base_patch16_224.in22k_ft_in22k',
 'beit_base_patch16_224.in22k_ft_in22k_in1k',
 'beit_base_patch16_384.in22k_ft_in22k_in1k',
 'beit_large_patch16_224.in22k_ft_in22k',
 'beit_large_patch16_224.in22k_ft_in22k_in1k',
 'beit_large_patch16_384.in22k_ft_in22k_in1k',
 'beit_large_patch16_512.in22k_ft_in22k_in1k',
 'beitv2_base_patch16_224.in1k_ft_in1k',
 'beitv2_base_patch16_224.in1k_ft_in22k',
 'beitv2_base_patch16_224.in1k_ft_in22k_in1k',
 'beitv2_large_patch16_224.in1k_ft_in1k',
 'beitv2_large_patch16_224.in1k_ft_in22k',
 'beitv2_large_patch16_224.in1k_ft_in22k_in1k',
 'botnet26t_256.c1_in1k',
 'caformer_b36.sail_in1k',
 'caformer_b36.sail_in1k_384',
 'caformer_b36.sail_in22k',
 'caformer_b36.sail_in22k_ft_in1k',
 'caformer_b36.sail_in22k_ft_in1k_384',
 'caformer_m36.sail_in1k',
 'caformer_m36.sail_in1k_384',
 'caformer_m36.sail_in22k',
 'caformer_m36.sail_in22k_ft_in1k',
 'caformer_m36.sail_in22k_ft_in1k_384',
 'caformer_s18.sail_in1k',
 'caformer_s18.s

In [ ]:
predictions = []
for fold in range(5):
    print('*'*25+f"Fold {fold}"+'*'*25)
    m_name = f'fold-{fold}'
    dls = get_dls(fold, 'imagenet')
    learn = vision_learner(dls, 'convnext_nano.in12k',
                           loss_func = CrossEntropyLossFlat(),
                           metrics=[accuracy, error_rate, Precision(average='macro'), Recall(average='macro'), RocAuc(average='macro')])
    learn.lr_find(suggest_funcs=(valley, slide))
    learn.recorder.plot_loss()
    learn.fine_tune(7, 0.01)

    interp = ClassificationInterpretation.from_learner(learn)
    losses,idxs = interp.top_losses()
    len(dls.valid_ds)==len(losses)==len(idxs)
    interp.plot_confusion_matrix(figsize=(7,7))

    valid = learn.dls.valid
    preds1 , targets = learn.tta(dl=valid, n=4)
    print(accuracy(preds1 , targets))

    test_dl = learn.dls.test_dl(trn_path + Test['filename'])
    preds1 , _ = learn.tta(dl=test_dl)
    preds1 = F.softmax(preds1, dim=1)
    predictions.append(preds1)

    free_memory([learn, preds1, dls])

*************************Fold 0*************************


epoch,train_loss,valid_loss,accuracy,error_rate,precision_score,recall_score,roc_auc_score,time
0,1.104099,1.058933,0.589375,0.410625,0.350614,0.388504,0.783427,07:34


epoch,train_loss,valid_loss,accuracy,error_rate,precision_score,recall_score,roc_auc_score,time
0,0.839174,0.798039,0.678366,0.321634,0.601463,0.427151,0.855507,08:13


In [ ]:
pred1 = 0
for i in range(5):
    pred1 += predictions[i]
pred1 /= 5

In [ ]:
submission_sf = pd.DataFrame({

    'ID': Test['ID'],
    'DR': preds1[:, 0].squeeze().numpy(),
    'G' : preds1[:, 1].squeeze().numpy(),
    'ND': preds1[:, 2].squeeze().numpy(),
    'WD': preds1[:, 3].squeeze().numpy(),
    'other': preds1[:, 4].squeeze().numpy(),
})
submission_sf.to_csv('convitfold.csv', index=False)

In [ ]:
submission_sf